In [1]:
from transformers import pipeline
import pandas as pd
import warnings
import ast
import os


warnings.filterwarnings('ignore') # for more clear output

c:\Users\saeed\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Reading dataset

In [2]:
path = "../instagram_data.csv"
data =pd.read_csv(path)
data.head()

,owner_id,owner_username,shortcode,is_video,caption,comments,likes,created_at,location,imageUrl,multiple_images,username,followers,following
0,36063641,christendominique,C3_GS1ASeWI,False,I’m a brunch & Iced Coffee girlie☕️🍳 \n\nTop @...,268,16382,1.709327e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t39....,True,christendominique,2144626.0,1021.0
1,36063641,christendominique,C38ivgNS3IX,True,😮‍💨Brow tips I really wish I would have know w...,138,9267,1.709241e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,christendominique,2144626.0,1021.0
2,36063641,christendominique,C35-Dd9SO1b,True,OMG I can’t believe it’s already been 1 yr sin...,1089,10100,1.709155e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,christendominique,2144626.0,1021.0
3,36063641,christendominique,C33TadDMisq,True,90’s Glam was Pam! \n\nMakeup \n@smashboxcosme...,271,6943,1.709065e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,christendominique,2144626.0,1021.0
4,36063641,christendominique,C3s-Cm1yCba,True,Chiseled & Sculptured 🏛️\n\n Contour @westmana...,145,17158,1.708718e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,christendominique,2144626.0,1021.0


In [3]:
data.shape

(11692, 14)

In [34]:
data.isna().sum()

owner_id              0
owner_username        2
shortcode             2
is_video              2
caption             161
comments              2
likes                 2
created_at            3
location           7169
imageUrl              3
multiple_images       3
username           2115
followers          2115
following          2115
dtype: int64

drop nan values in caption column

In [4]:
data.dropna(subset=["caption", "username"], inplace=True)

see if there is any diffrance in user names on two column for each row 
there are some cases but just when username is NAN


In [5]:
data.loc[(data.owner_username != data.username) & (data.username.isna() != True)]

,owner_id,owner_username,shortcode,is_video,caption,comments,likes,created_at,location,imageUrl,multiple_images,username,followers,following


In [6]:
data.isna().sum()

owner_id              0
owner_username        0
shortcode             0
is_video              0
caption               0
comments              0
likes                 0
created_at            0
location           5697
imageUrl              0
multiple_images       0
username              0
followers             0
following             0
dtype: int64

-------

# Language detection 

I want to perform sentiment analysis on captions in this dataset. For increased accuracy, I want to translate all captions to English. Before that, we must detect the language of each caption and label them accordingly.

for this detection i will use `transform` library 

In [38]:
data.head(1)

,owner_id,owner_username,shortcode,is_video,caption,comments,likes,created_at,location,imageUrl,multiple_images,username,followers,following
0,36063641,christendominique,C3_GS1ASeWI,False,I’m a brunch & Iced Coffee girlie☕️🍳 \n\nTop @...,268,16382,1.709327e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t39....,True,christendominique,2144626.0,1021.0


In [57]:
def language_detection(row):

    text = row["caption"]
    model_ckpt = "papluca/xlm-roberta-base-language-detection"
    pipe = pipeline("text-classification", model=model_ckpt)
    result = pipe(text, top_k=1, truncation=True)
    return result[0]["label"]

In [59]:
data

,owner_id,owner_username,shortcode,is_video,caption,comments,likes,created_at,location,imageUrl,multiple_images,username,followers,following
0,36063641,christendominique,C3_GS1ASeWI,False,I’m a brunch & Iced Coffee girlie☕️🍳 \n\nTop @...,268,16382,1.709327e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t39....,True,christendominique,2144626.0,1021.0
1,36063641,christendominique,C38ivgNS3IX,True,😮‍💨Brow tips I really wish I would have know w...,138,9267,1.709241e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,christendominique,2144626.0,1021.0
2,36063641,christendominique,C35-Dd9SO1b,True,OMG I can’t believe it’s already been 1 yr sin...,1089,10100,1.709155e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,christendominique,2144626.0,1021.0
3,36063641,christendominique,C33TadDMisq,True,90’s Glam was Pam! \n\nMakeup \n@smashboxcosme...,271,6943,1.709065e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,christendominique,2144626.0,1021.0
4,36063641,christendominique,C3s-Cm1yCba,True,Chiseled & Sculptured 🏛️\n\n Contour @westmana...,145,17158,1.708718e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,christendominique,2144626.0,1021.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11686,877755538,m_eye_nd,C35xLDsPPcj,True,which part is it? 🥺💛🫂,0,123,1.709148e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t51....,False,m_eye_nd,331373.0,260.0
11687,877755538,m_eye_nd,C35pT7pPs8e,False,which one is your fav 🥺💛🌌🌊🥀,4,1241,1.709144e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t39....,False,m_eye_nd,331373.0,260.0
11688,877755538,m_eye_nd,C34fjtCvi45,False,do you miss that? 🥺💛😭,11,806,1.709105e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t39....,False,m_eye_nd,331373.0,260.0
11689,877755538,m_eye_nd,C33nKF4vCQR,False,"but it is wonderful while it lasts, right? 🥺💛",15,4828,1.709076e+09,NaN,https://instagram.flba2-1.fna.fbcdn.net/v/t39....,False,m_eye_nd,331373.0,260.0


In [58]:
data["caption_language"] = data.apply(language_detection, axis=1)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.
Some weights of the PyTorch model were not used when initializing 

MemoryError: Unable to allocate 732. MiB for an array with shape (250002, 768) and data type float32

In [ ]:
data.head(1)

In [ ]:
folder_path = '../2.Data_with_detected_caption_language'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Save each DataFrame to a CSV file in the new folder
data.to_csv(os.path.join(folder_path, 'Data_with_detected_caption_language.csv'), index=False)

---

# Locational Data
data rows with filled location values

In [7]:
locational_data = data.loc[data.location.isna() != True]
locational_data.head(1)

,owner_id,owner_username,shortcode,is_video,caption,comments,likes,created_at,location,imageUrl,multiple_images,username,followers,following
28,40506412,christinehmcconnell,CzJtDjegmdI,False,Had an amazing time in #salem for Halloween th...,466,79000,1.698945e+09,"{'id': '34537076', 'has_public_page': True, 'n...",https://instagram.ffor43-1.fna.fbcdn.net/v/t39...,True,christinehmcconnell,597189.0,1576.0


*`coverting all values to dictionary`*

In [8]:
def dict_maker(row):
    return ast.literal_eval(row["location"])

locational_data["location"] = locational_data.apply(dict_maker, axis=1)

*`getting all city names in dictionary, and then replace the final list of city names with current column of location`*

In [9]:
location = []
for key in locational_data.location:
    location.append(key["slug"])

# replace old colum with new list
locational_data.location = location

In [10]:
locational_data = locational_data[locational_data['username'].notna()]
locational_data.head(1)

,owner_id,owner_username,shortcode,is_video,caption,comments,likes,created_at,location,imageUrl,multiple_images,username,followers,following
28,40506412,christinehmcconnell,CzJtDjegmdI,False,Had an amazing time in #salem for Halloween th...,466,79000,1.698945e+09,salem-massachusetts,https://instagram.ffor43-1.fna.fbcdn.net/v/t39...,True,christinehmcconnell,597189.0,1576.0


i will convert this to dataset "Locational_dataset" 

In [11]:
locational_data.isna().sum()

owner_id           0
owner_username     0
shortcode          0
is_video           0
caption            0
comments           0
likes              0
created_at         0
location           0
imageUrl           0
multiple_images    0
username           0
followers          0
following          0
dtype: int64

In [12]:
locational_data.shape

(3744, 14)

----

# Non-Locational Data

In [24]:
nonlocational_data = data.loc[data.username.isna() == False]
nonlocational_data.drop(columns="location", inplace=True)

In [25]:
nonlocational_data.head(1)

,owner_id,owner_username,shortcode,is_video,caption,comments,likes,created_at,imageUrl,multiple_images,username,followers,following
0,36063641,christendominique,C3_GS1ASeWI,False,I’m a brunch & Iced Coffee girlie☕️🍳 \n\nTop @...,268,16382,1.709327e+09,https://instagram.flba2-1.fna.fbcdn.net/v/t39....,True,christendominique,2144626.0,1021.0


In [26]:
nonlocational_data.isna().sum()

owner_id           0
owner_username     0
shortcode          0
is_video           0
caption            0
comments           0
likes              0
created_at         0
imageUrl           0
multiple_images    0
username           0
followers          0
following          0
dtype: int64

---

# Exporting Data 
Exporting Data on csv file in `prepare data` folder

In [27]:
folder_path = '../2.Prepared Data'
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Save each DataFrame to a CSV file in the new folder
nonlocational_data.to_csv(os.path.join(folder_path, 'non_locational_data.csv'), index=False)
locational_data.to_csv(os.path.join(folder_path, 'locational_data.csv'), index=False)

PermissionError: [Errno 13] Permission denied: '../2.Prepared Data\\non_locational_data.csv'